In [ ]:
import numpy as np
import datetime
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from pybob.GeoImg import GeoImg
from pybob.bob_tools import bin_data
from pybob import image_tools as it, ddem_tools as dt, plot_tools as pt
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA

In [ ]:
def fill_column(col, newsize):
    dsize = newsize - col.size
    return np.concatenate([col, np.nan*np.zeros(dsize)])

In [ ]:
font = {'family': 'sans',
        'weight': 'normal',
        'size': 24}
legend_font = {'family': 'sans',
               'weight': 'normal',
               'size': '18'}
matplotlib.rc('font', **font)
plt.ion()

boxprops = dict(linestyle='-', linewidth=2, color='k')
medianprops = dict(linestyle='-', linewidth=2, color='k')

In [ ]:
method_markers = {'Mean dH': ('x', 'purple'), 
                  'Med. dH': ('+', 'purple'),
                  'dH interp.': ('d', 'r'),
                  'Z interp.': ('o', 'r'),
                  '1km neighborhood': ('s', 'r'),
                  'Glob. Mean Hyps.': ('^', 'k'),
                  'Glob. Med. Hyps.': ('*', 'k'),
                  'Glob. Poly. Hyps.': ('.', 'k'),
                  'Loc. Mean Hyps.': ('^',  'b'),
                  'Loc. Med. Hyps.': ('*', 'b'),
                  'Loc. Poly. Hyps.': ('.', 'b')}

label_names = {'Mean dH': 'const. mean', 
               'Med. dH': 'const. med.',
               'dH interp.': 'lin. interp. dH',
               'Z interp.': 'lin. interp. Z',
               '1km neighborhood': '1km avg.',
               'Glob. Mean Hyps.': 'Glob. Mean Hyps.',
               'Glob. Med. Hyps.': 'Glob. Med. Hyps.',
               'Glob. Poly. Hyps.': 'Glob. Poly. Hyps.',
               'Loc. Mean Hyps.': 'Loc. Mean Hyps.',
               'Loc. Med. Hyps.': 'Loc. Med. Hyps.',
               'Loc. Poly. Hyps.': 'Loc. Poly. Hyps.'}

## load images, data files

In [ ]:
gmask = '../outlines/01_rgi60_Alaska_GlacierBay_02km_UTM.shp'
glac_outlines = gpd.read_file(gmask)
glac_outlines.set_index('RGIId', inplace=True)

void_list = [35, 50, 70, 80, 90, 95]

df = pd.read_csv('volume_change_results.csv')

common_cols = ['base dV', 'Area', 'uncert.', 'dt']
dv_cols = ['Mean dH', 'Med. dH', 'dH interp.', 'Z interp.', '1km neighborhood', 'Glob. Mean Hyps.',
           'Glob. Med. Hyps.', 'Glob. Poly. Hyps.', 'Loc. Mean Hyps.', 'Loc. Med. Hyps.', 'Loc. Poly. Hyps.']
df.set_index('RGIId', inplace=True)

df['base dV'] = df['base dV'] / df['Area'] / df['dt'] # base dV in m/a
df['uncert.'] = df['uncert.'] / df['Area'] / df['dt'] # uncert. in m/a

for c in dv_cols:
    for v in void_list:
        df[c + str(v)] = df[c + str(v)] / df['Area'] / df['dt'] # dV estimates in m/a
df.dropna(inplace=True)

### plot comparison of interp. volume change vs measured

Figure 5 from the paper, plots a direct comparison of the true and estimated volume change for the 50% correlation threshold.

In [ ]:
v = 50 # change this to see other thresholds (i.e., 35, 70, 80,...)
plt.figure(facecolor='w', figsize=(12, 12), dpi=200)
plt.plot(range(-10, 3), range(-10, 3), 'k--')

for c in dv_cols:
    if method_markers[c][0] == '.':
        fs = 'full'
    else:
        fs = 'none'
    plt.plot(df['base dV'], df[c + str(v)], method_markers[c][0], 
             color=method_markers[c][1], ms=18, label=label_names[c].lower(), 
             alpha=0.35, fillstyle=fs)

ax = plt.gca()
ax.legend(loc='lower right', prop=legend_font)


plt.xlabel('true volume change (m a$^{-1}$)')
plt.ylabel('void-filled volume change (m a$^{-1}$)')

ax.axis('scaled')
ax.set_xlim(-7, 1)
ax.set_ylim(-7, 1)

The rest of the analysis is done with the differences to the true values, so we subtract out the true volume change from each column. We also assume that any differences beyond 10 m/a are outliers, and remove them from the analysis.

In [ ]:
for v in void_list:
    for c in dv_cols:
        df[c + str(v)] = df[c + str(v)] - df['base dV']
        df[c + str(v)][np.abs(df[c + str(v)]) > 10] = np.nan
df.dropna(inplace=True)

## Table 1
Print Table 1 from the paper, in TeX format.

In [ ]:
v = 50 # change this to see other thresholds (i.e., 35, 70, 80,...)
print('method  & mean $\pm$ std & median & max & min & rms diff & total diff & pct. uncert. \\\\\hline')
for c in dv_cols:
    mu = df[c + str(v)].mean()
    sig = df[c + str(v)].std()
    med = df[c + str(v)].median()
    max_diff = np.max(df[c + str(v)])
    min_diff = np.min(df[c + str(v)])
    rms = np.sqrt(np.nansum(df[c + str(v)]**2) / np.count_nonzero(~np.isnan(df[c + str(v)])))
    total_diff = np.nansum(df[c + str(v)] * df['Area']) / 1e9
    pct_uncert = 100 * np.count_nonzero(np.abs(df[c + str(v)]) < df['uncert.']) / df['uncert.'].size
    #total_diff = np.nansum(df[c + str(v)])
    print('{} & {:.2f} $\pm$ {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\'.format(label_names[c].lower(),
                                                                                   mu, sig, med,
                                                                                   max_diff, min_diff, 
                                                                                   rms, total_diff, pct_uncert))

Sort the glaciers by area (1-10 km2, 10-50 km2, 50-100 km2, over 100 km2), and show box plots for each grouping. For glaciers over 100 km2, we show their spread as individuals.

In [ ]:
v = 50 # change this to see other thresholds (i.e., 35, 70, 80,...)
#df = df_list[i]
binned = np.digitize(df['Area'], 1e6 * np.array([0, 10, 50, 100]))

grouped = []
for b in np.unique(binned):
    grouped.append(df[binned == b])

smallest_dV = np.nansum(grouped[0]['base dV'] * grouped[0]['Area']) / grouped[0]['Area'].sum()
medium_dV = np.nansum(grouped[1]['base dV'] * grouped[1]['Area']) / grouped[1]['Area'].sum()
largest_dV = np.nansum(grouped[2]['base dV'] * grouped[2]['Area']) / grouped[2]['Area'].sum()

smallest = np.empty(0)
medium = np.empty(0)
largest = np.empty(0)

for c in dv_cols:
    smallest = np.concatenate([smallest, grouped[0][c+ str(v)]])
    medium = np.concatenate([medium, grouped[1][c+ str(v)]])
    largest = np.concatenate([largest, grouped[2][c+ str(v)]])

medium = fill_column(medium, smallest.size)
largest = fill_column(largest, smallest.size)

grouped_data = np.vstack([smallest, medium, largest]).transpose()
newdf = pd.DataFrame(grouped_data, columns=['1-10 km$^2$ (n={})'.format(grouped[0].shape[0]),
                                            '10-50 km$^2$ (n={})'.format(grouped[1].shape[0]),
                                            '50-100 km$^2$ (n={})'.format(grouped[2].shape[0])])

sorted_3 = grouped[3].sort_values('Area', ascending=True)
these_cols = [c + str(v) for c in dv_cols]
ind_data = sorted_3[these_cols].values
dVs = np.hstack([sorted_3['base dV'].values, largest_dV, medium_dV, smallest_dV])

dsize = smallest.size - ind_data.shape[1]
add_data = np.zeros((20, dsize)) + np.nan
ind_data_filled = np.hstack([ind_data, add_data]).transpose()

pretty_cols = [c.split('.')[1] for c in sorted_3.index[:-1]]
ind_df = pd.DataFrame(ind_data_filled, columns=pretty_cols + [sorted_3.index[-1]])

final_df = newdf.join(ind_df)
f = plt.figure(facecolor='w', figsize=(15, 15), dpi=200)
bp = final_df.boxplot(vert=False, return_type='dict')

for key in bp.keys():
    for item in bp[key]:
        item.set_linewidth(2)
        item.set_color('k')

ax = plt.gca()
ax.set_xlim(-1.5, 1.5)

yloc = 22.7
ax.text(1.52, yloc, '{: .2f} '.format(dVs[0]) + 'm a$^{-1}$')
for dv in dVs[1:]:
    yloc -= 1
    ax.text(1.52, yloc, '{: .2f} '.format(dv))

yloc = 3.7
for a in sorted_3['Area'].values:
    ax.text(-1.5, yloc, '{: .0f} km$^2$'.format(a / 1e6))
    yloc += 1

plt.xlabel('difference to truth (m a$^{-1}$)')

## Figure 7

Figure 7 from the paper shows how each method did relative to the the uncertainty estimates for each glacier over 100 km2. Gray bars indicate uncertainty values in m/a.

In [ ]:
y_inds = np.arange(0, sorted_3.index.size, 1)
f = plt.figure(facecolor='w', figsize=(15, 15), dpi=200)
v = 50 # 35, 50, 70, 80, 90, 95
plt.plot([0, 0], [-1, 20], 'k--', linewidth=2)
errs = []
for j in y_inds:
    y = j - 0.5
    x = -sorted_3['uncert.'][j]
    errs.append(matplotlib.patches.Rectangle((x, y), 2*sorted_3['uncert.'][j], 1, facecolor='0.5', edgecolor='k'))
    
p = matplotlib.collections.PatchCollection(errs, alpha=0.4, facecolor='0.5', edgecolor='k')
plt.gca().add_collection(p)   
plt.draw()

for c in dv_cols:
    if method_markers[c][0] == '.':
        fs = 'full'
    else:
        fs = 'none'
    plt.plot(sorted_3[c + str(v)], y_inds, method_markers[c][0], color=method_markers[c][1],
             ms=18, label=label_names[c].lower(), fillstyle=fs)

plt.gca().set_yticks(y_inds)
plt.gca().set_yticklabels(pretty_cols + [sorted_3.index[-1]])
plt.grid(axis='y')
plt.xlim(-2, 2)
plt.ylim(-0.5, 19.5)
plt.legend(loc='lower left', prop=legend_font)

plt.xlabel('difference to truth (m a$^{-1}$)')

## Figure 8

Figure 8 shows box plots of the difference to truth for glaciers over 100 km2, and all glaciers.

In [ ]:
fig8 = plt.figure(facecolor='w', figsize=(16, 8), dpi=200)

ax = fig8.add_subplot(111)    # The big subplot
ax1 = fig8.add_subplot(121)
ax2 = fig8.add_subplot(122)

# Turn off axis lines and ticks of the big subplot
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['right'].set_color('none')
ax.tick_params(labelcolor='w', top='off', bottom='off', left='off', right='off')

v = 50
#new_cols = [c.lower().replace('70', '') for c in grouped[3].columns]
#grouped[3].columns = new_cols
these_cols = [c + str(v) for c in dv_cols[::-1]]
new_grp = pd.concat(grouped[0:1])
bp1 = grouped[3][these_cols].boxplot(vert=False, return_type='dict', ax=ax1)
#bp = grouped[0][these_cols].boxplot(vert=False, return_type='dict')
bp2 = new_grp[these_cols].boxplot(vert=False, return_type='dict', ax=ax2)

for key in bp1.keys():
    for item in bp1[key]:
        item.set_linewidth(2)
        item.set_color('k')
for key in bp2.keys():
    for item in bp2[key]:
        item.set_linewidth(2)
        item.set_color('k')

ax1.set_xlim(-2, 2)
ax.set_xlabel('difference to truth (m a$^{-1}$)')
ax1.set_yticklabels([label_names[c.replace(str(v), '')].lower() for c in these_cols])
ax1.text(-1.9, 10.7, 'a', fontsize=36)

ax2.set_xlim(-2, 2)
ax2.set_yticklabels([])
ax2.text(-1.9, 10.7, 'b', fontsize=36)

This figure shows the same as above, but for a single area grouping (1-10 km2, 10-50 km2, 50-100 km2, or over 100 km2)

In [ ]:
plt.figure(facecolor='w', figsize=(12, 12), dpi=200)
v = 50 # correlation threshold
these_cols = [c + str(v) for c in dv_cols]
# grouped[0] is glaciers under 10 km2, grouped[3] is glaciers over 100 km2
bp = grouped[1][these_cols].boxplot(vert=False, return_type='dict')

for key in bp.keys():
    for item in bp[key]:
        item.set_linewidth(2)
        item.set_color('k')
ax = plt.gca()
ax.set_xlim(-2, 2)
ax.set_xlabel('difference to truth (m a$^{-1}$)')
ax.set_yticklabels([c.lower().replace(str(v), '') for c in these_cols])

This plot shows how the values change for each large glacier when going from a 50% correlation threshold to a 70% correlation threshold.

In [ ]:
for c in dv_cols:
    sorted_3_70[c + '70'] = sorted_3_70[c + '70'] - sorted_3[c + '50']

y_inds = np.arange(0, sorted_3.index.size, 1)
f = plt.figure(facecolor='w', figsize=(15, 15), dpi=200)

# plot the 70% void data
v = 70
plt.plot([0, 0], [-1, 20], 'k--', linewidth=2)
plt.plot(sorted_3_70['Mean dH{}'.format(v)], y_inds, 'x',
         label='mean dH', ms=18, color='purple')
plt.plot(sorted_3_70['Med. dH{}'.format(v)], y_inds, '+',
         label='med. dH', ms=18, color='purple')

plt.plot(sorted_3_70['dH interp.{}'.format(v)], y_inds, 'd',
         label='dH int.', ms=18, color='r', fillstyle='none')
plt.plot(sorted_3_70['Z interp.{}'.format(v)],  y_inds,'o',
         label='Z int.', ms=18, color='r', fillstyle='none')
plt.plot(sorted_3_70['1km neighborhood{}'.format(v)], y_inds, 's',
         label='1km avg.', ms=18, color='r', fillstyle='none')

plt.plot(sorted_3_70['Glob. Mean Hyps.{}'.format(v)], y_inds, '^',
         label='glob. mean hyps.', ms=18, color='k', fillstyle='none')
plt.plot(sorted_3_70['Glob. Med. Hyps.{}'.format(v)], y_inds, '*',
         label='glob. med. hyps.', ms=18, color='k', fillstyle='none')
plt.plot(sorted_3_70['Glob. Poly. Hyps.{}'.format(v)], y_inds, '.',
         label='glob. poly. hyps.', ms=18, color='k')

# local methods: use squares
plt.plot(sorted_3_70['Loc. Mean Hyps.{}'.format(v)], y_inds, '^',
         label='loc. mean hyps.', ms=18, color='b', fillstyle='none')
plt.plot(sorted_3_70['Loc. Med. Hyps.{}'.format(v)], y_inds, '*',
         label='loc. med. hyps.', ms=18, color='b', fillstyle='none')
plt.plot(sorted_3_70['Loc. Poly. Hyps.{}'.format(v)], y_inds, '.',
         label='loc. poly. hyps.', ms=18, color='b')

plt.legend(loc='lower left', prop=legend_font)
plt.gca().set_yticks(y_inds)
plt.gca().set_yticklabels(sorted_3.index)
plt.grid(axis='y')
plt.xlim(-0.5, 0.5)
plt.ylim(-0.5, 19.5)

plt.xlabel('difference to 50% threshold (m a$^{-1}$)')

## Figure 12

Figure 12 from the paper shows how the three "best" methods perform as a function of percent void. This figure takes the values for all correlation thresholds and sorts them by percent void.

In [ ]:
grp_df_list = []
for v in void_list:
    this_df = pd.DataFrame()
    this_df['Pct Void'] = np.floor(df['Pct Void' + str(v)] / 0.08) * 0.08
    for c in dv_cols:
        this_df[c] = df[c + str(v)]
    grp_df_list.append(this_df)

grp_df = pd.concat(grp_df_list)
stats_df = grp_df[dv_cols].groupby(grp_df['Pct Void']).describe()

plt.figure(figsize=(15,15), facecolor='w', dpi=200)

cols = ['dH interp.', 'Loc. Mean Hyps.', 'Glob. Mean Hyps.']
for c in cols:
    valid = stats_df[c]['count'] >= 4
    this_mean = stats_df[c]['mean'][valid]
    this_std = stats_df[c]['std'][valid]
    this_up = this_mean + this_std
    this_dn = this_mean - this_std
    this_voids = stats_df.index.values[valid]

    plt.plot(100 * (this_voids + 0.04), this_mean, method_markers[c][1],
             linewidth=2, label=label_names[c].lower())
    env_x = 100 * np.hstack([this_voids, 1, 1, np.flipud(this_voids), 0])
    env_y = np.hstack([this_dn, this_dn.values[-1], this_up.values[-1], np.flipud(this_up), this_up.values[0]])
    patchcoords = np.column_stack([env_x, env_y])
    env_patch = Polygon(patchcoords, closed=True, facecolor=method_markers[c][1], alpha=0.15)
    plt.gca().add_patch(env_patch)

plt.xlim(0, 100)
plt.ylim(-1, 1)

plt.ylabel('difference to truth (m a$^{-1}$)')
plt.xlabel('percent void')
plt.legend(loc='upper left')

This figure is the same as above, but only for a given correlation threshold.

In [ ]:
v = 90
void_grp = np.floor(df['Pct Void' + str(v)] / 0.05) * 0.05
df_cols = [c + str(v) for c in dv_cols]
df['void_grp'] = void_grp
grp_df = df[df_cols].groupby(df['void_grp']).describe()

plt.figure(figsize=(15,15))

cols = ['dH interp.', 'Loc. Mean Hyps.', 'Glob. Mean Hyps.']
for c in cols:
    valid = grp_df[c + str(v)]['count'] >= 4
    this_mean = grp_df[c + str(v)]['mean'][valid]
    this_std = grp_df[c + str(v)]['std'][valid]
    this_up = this_mean + this_std
    this_dn = this_mean - this_std
    this_voids = grp_df.index.values[valid]

    plt.plot(100 * this_voids, this_mean, method_markers[c][1], linewidth=2, label=c)

    env_x = 100 * np.hstack([this_voids, np.flipud(this_voids)])
    env_y = np.hstack([this_dn, np.flipud(this_up)])
    patchcoords = np.column_stack([env_x, env_y])
    env_patch = Polygon(patchcoords, closed=True, facecolor=method_markers[c][1], alpha=0.15)
    plt.gca().add_patch(env_patch)

plt.xlim(0, 100)
plt.ylim(-1, 1)

plt.ylabel('difference to truth (m a$^{-1}$)')
plt.xlabel('percent void')
plt.legend(loc='upper left')

Plot the percentage of estimates that fall outside of the original uncertainty range for the top 5 methods (constant mean, interpolation of elevation chagne, global mean hypsometric, local mean and median hypsometric).

In [ ]:
grp_df_list = []
for v in void_list:
    this_df = pd.DataFrame()
    void_grp = np.floor(df['Pct Void' + str(v)] / 0.06) * 0.06
    this_df['Pct Void'] = void_grp
    for c in dv_cols:
        col = np.zeros(void_grp.shape)
        inside = np.abs(df[c + str(v)]) < df['uncert.']
        col[inside] = 1
        this_df[c] = col
    grp_df_list.append(this_df)

grp_df = pd.concat(grp_df_list)
sum_df = grp_df[dv_cols].groupby(grp_df['Pct Void']).sum()
des_df = grp_df[dv_cols].groupby(grp_df['Pct Void']).describe()

count = des_df['Mean dH']['count']
valid = count > 5
void_grps = sum_df.index.values

plt.figure(figsize=(12,12), facecolor='w', dpi=200)

cols = ['Mean dH', 'dH interp.', '1km neighborhood', 'Glob. Mean Hyps.',
        'Loc. Mean Hyps.', 'Loc. Med. Hyps.']
for c in cols:
    if method_markers[c][0] == '.':
        fs = 'full'
    else:
        fs = 'none'
    plt.plot(100 * (void_grps[valid] + 0.03), 100 * sum_df[c][valid] / count[valid],
             method_markers[c][0], color=method_markers[c][1], ms=14,
             label=label_names[c].lower(), linewidth=10, fillstyle=fs)

plt.xlabel('percent void')
plt.ylabel('percent less than uncertainty')

plt.gca().set_xlim(0, 102)
plt.gca().set_ylim(0, 102)
#plt.legend(loc='upper center', bbox_to_anchor=(1.2, 1.01), prop=legend_font)
plt.legend()

Plot the mean (± std. dev) percentage of voids per glacier for each correlation threshold.

In [ ]:
plt.figure(figsize=(10,10), facecolor='w')

voids = [c for c in df.columns if 'Void' in c]
voids.sort()

mean_voids = df[voids].mean()
std_voids = df[voids].std()
this_up = mean_voids + std_voids
this_dn = mean_voids - std_voids

plt.plot(void_list, 100 * df[voids].mean(), 'k')

env_x = np.hstack([void_list, 95, 95, np.flipud(void_list), 35])
env_y = 100 * np.hstack([this_dn, this_dn.values[-1], this_up.values[-1], np.flipud(this_up), this_up.values[0]])

patchcoords = np.column_stack([env_x, env_y])
env_patch = Polygon(patchcoords, closed=True, facecolor='k', alpha=0.15)
plt.gca().add_patch(env_patch)
plt.gca().set_xlim(0, 100)
plt.gca().set_ylim(0, 100)

plt.xlabel('correlation threshold')
plt.ylabel('mean percent void')

## Table 2

Table 2 from the papers shows the results for Taku and Field Glaciers, which have the largest differences from truth in terms of total volume.

In [ ]:
v = 50 # correlation threshold
taku_data = df.loc[['RGI60-01.01390']]
field_data = df.loc[['RGI60-01.01520']]

print('method & Taku Glacier & Field Glacier \\\\\hline')
for c in dv_cols:
    print('{} & {:.2f} & {:.2f} \\\\'.format(label_names[c].lower(), taku_data[c + str(v)].values[0],
                                             field_data[c + str(v)].values[0]))

print('pct void & {:.2f} & {:.2f} \\\\'.format(100 * taku_data['Pct Void' + str(v)].values[0],
                                               100 * field_data['Pct Void' + str(v)].values[0]))

### Field Glacier comparison
Show how taking the median elevation difference is probably a bad idea.

In [ ]:
v = 50 # correlation threshold
ifsar = GeoImg('2013/seak.ifsar.2013.dem.30m_adj.tif')
ifsar_srtm = GeoImg('2013/ifsar_srtm_2013_dh.tif')
srtm = GeoImg('2013/SRTM_SE_Alaska_30m_2013IfSAR_adj.tif')

aad_srtm = GeoImg('hypsometries/IfSAR_AAD_DEM_2013.tif')

mask_full = GeoImg('../southeast_average_corr.tif')
mask_sub = mask_full.reproject(ifsar)

holy_ifsar_srtm = ifsar_srtm.copy()
holy_ifsar_srtm.img[mask_sub.img < v] = np.nan

gmask_tif = GeoImg('GlacierBay_Mask_2013.tif')
glacier_mask = gmask_tif.img
glacs = np.unique(gmask_tif.img[np.isfinite(gmask_tif.img)])

glac_shp = gpd.read_file('../outlines/01_rgi60_Alaska_GlacierBay_02km_UTM_2013.shp')

In [ ]:
field_mask = glacier_mask == 873 # this index may have changed - check shapefile!
field_void_dH = holy_ifsar_srtm.img[field_mask]
field_nonvoid_dH = ifsar_srtm.img[field_mask]

bins = np.arange(-100, 25, 1)
plt.figure(facecolor='w', figsize=(10, 5), dpi=200)
n1, b1, p1 = plt.hist(field_void_dH[np.isfinite(field_void_dH)], bins, 
                      alpha=0.75, normed=True, label='voided', facecolor='k', edgecolor='k')
n2, b2, p2 = plt.hist(field_nonvoid_dH[np.isfinite(field_nonvoid_dH)], bins, 
                      alpha=0.8, normed=True, label='non-voided', facecolor='0.9', edgecolor='k')
plt.plot([np.nanmean(field_void_dH), np.nanmean(field_void_dH)], [0, 0.1], '-.', color='0.5',
         ms=20, label='voided mean', linewidth=2)
plt.plot([np.nanmedian(field_void_dH), np.nanmedian(field_void_dH)], [0, 0.1], 'k--', 
         ms=20, label='voided median', linewidth=2)
# plt.plot(np.nanmean(field_nonvoid_dH), 0, 'r^', ms=12, label='non-voided mean')
# plt.plot(np.nanmedian(field_nonvoid_dH), 0, 'r*', ms=12, label='non-voided median')
plt.xlabel('elevation change (m)')
plt.ylabel('frequency')
plt.legend(prop=legend_font)

plt.xlim(-100, 25)
plt.ylim(0, 0.1)

## total regional volume change and uncertainty

Calculates the total "regional" volume change (by summing all of the glaciers for which we have values), and estimates the uncertainty based on the off-glacier differences in the DEMs as well as the co-registration with ICESat.

In [ ]:
r = 30 # pixel size
L = 500 # assumed autocorrelation length
eps_a = 0.1 # assumed 10% area uncertainty
eps_rand = 8.71 # comes from the off-glacier statistics
eps_bias = 1.32 # comes from co-registration with ICESat

dv = np.sum(df['base dV'] * df['Area']) # total in m3/yr
area = df['Area'].sum() # total glacier area

dh = dv / area # mean elevation change
n_pix = area / r / r # number of pixels

# eqn. 2 from the paper - volume change uncertainty due to dh uncertainty
e_h = np.sqrt(eps_rand**2 / (np.sqrt(n_pix/(L/r)**2)) + eps_bias**2) * area
# volume change uncertainty due to area uncertainty
e_a = dv * eps_a

# eqn. 3 from the paper - total uncertainty in volume change
total_uncert = np.sqrt(e_h**2 + e_a**2) / 13.1 / area # total in m
print('Total volume change for region: {:.2f}±{:.2f} m/yr'.format(dh, total_uncert))

## volume change vs area

Plot volume change vs. area for all glaciers.

In [ ]:
plt.figure(facecolor='w', figsize=(8,8))

plt.plot(df['Area'] / 1e6, df['base dV'], 'o')
plt.plot([0, 600], [df['base dV'].mean(), df['base dV'].mean()], 'k--')

plt.xlabel('area (km$^2$)')
plt.ylabel('volume change (m a$^{-1}$)')
plt.gca().set_xlim(0, 600)